**OPENAI API PROMPT SETUP**

This notebook makes use of the Chatgpt API and prompts this LLM to asks for the political stance in Mastodon Toots. Notebook can be altered to use different dataset or different prompts.

In [ ]:
%pip install pandas langchain openai
    

In [81]:
pip install openai==0.28


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/76.5 kB ? eta -:--:--
   ------------------------------------- -- 71.7/76.5 kB 787.7 kB/s eta 0:00:01
   ---------------------------------------- 76.5/76.5 kB 608.3 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.1
    Uninstalling openai-1.57.1:
      Successfully uninstalled openai-1.57.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.10 requires openai<2.0.0,>=1.54.0, but you have openai 0.28.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Importing dependencies
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field
import pandas as pd
import os
import openai
import time

C:\Users\rickv\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# Initialize the API with your own key, Key will not be published since confidential information is related to this key. 
OPENAI_KEY= 'placeholder'
OPENAI_API_VERSION='2024-02-15-preview'
MODEL="gpt-3.5-turbo-16k"
MODEL_TEMPERATURE=0.0

In [3]:
# LLM is initalized with variables stated above.
llm = ChatOpenAI(
    model=MODEL,
    temperature=0,
    api_key = OPENAI_KEY
)

AttributeError: module 'openai' has no attribute 'OpenAI'

In [4]:
# Load CSV file
file_path = "All_toots_cleaned.csv"  # Replace with your CSV file path
df = pd.read_csv(file_path)

In [43]:
# Limit processing to the first 10 entries TEST !!
# df_limited = df.head(60)


In [66]:
# System message to set the task for the model
system_message = SystemMessage(
    content=(
        "You are a political analyst. Based on the given message content, classify the political standpoint strictly as one of the following: "
        "'left', 'right', 'center' or 'N/A' if the classification is unclear. Respond with only one of these words."
    )
)

In [67]:
# Function to classify political standpoint
def classify_political_standpoint(Content):
    messages = [
        system_message,
        HumanMessage(content=f"Message: {Content}\n\nWhat is the political standpoint?")
    ]
    response = llm(messages)
    return response.content.strip()


In [1]:
# Process the messages and classify their political standpoint
df['political_standpoint'] = df['Content'].apply(classify_political_standpoint)

In [59]:
# print(df['political_standpoint'])

In [ ]:
folder_path = ''
df_limited.to_csv(os.path.join(folder_path, "All_toots_processed.csv"), index=False)

In [ ]:
folder_path = ''
df.to_csv(os.path.join(folder_path, "All_toots_processed1.csv"), index=False)

In [16]:
# Code to run trough the entire dataset that is provided in batches to reduce API token costs.
openai.api_key = 'placeholder'

# Function to classify political standpoint for a single content
def classify_political_standpoint(content):
    system_message = {
        "role": "system", 
        "content": "You are a political analyst. Based on the given message content, classify the political standpoint strictly as one of the following: 'left', 'right', 'center' or 'N/A' if the classification is unclear. Respond with only one of these words. Please try to steer clear as much as possible of the answer N/A."
    }

    user_message = {
        "role": "user", 
        "content": f"Message: {content}\n\nWhat is the political standpoint?"
    }

    messages = [system_message, user_message]

    try:
        # Call the OpenAI API for classification
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",  # Adjust model version as needed
            messages=messages
        )
        return response['choices'][0]['message']['content'].strip()  # Extract the classification result
    except openai.error.RateLimitError:
        print("Rate limit hit. Sleeping for 10 seconds...")
        time.sleep(10)  # Sleep and retry if rate-limited
        return classify_political_standpoint(content)  # Retry after waiting
    except Exception as e:
        print(f"An error occurred: {e}")
        return "N/A"  # Return N/A if there is an error

# Load your CSV file
file_path = "all_toots_cleaned.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Iterate through each content entry one by one and classify
political_standpoints = []
for content in df['Content']:
    standpoint = classify_political_standpoint(content)
    political_standpoints.append(standpoint)

    
# Add results to DataFrame
df['political_standpoint'] = political_standpoints

# Save the DataFrame back to CSV
df.to_csv("all_toots_with_standpoint_V2.csv", index=False)

# Show the first few rows with the new column
print(df.head())

An error occurred: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_72c66423f1b50dcc52b5eacc8dad2a0a in your email.) {
  "error": {
    "message": "The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_72c66423f1b50dcc52b5eacc8dad2a0a in your email.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
} 500 {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_72c66423f1b50dcc52b5eacc8dad2a0a in your email.)', 'type': 'server_error', 'param': None, 'code': N